# Understanding Python Annotations


## Python Annotations
- Introduced in [PEP 3107](https://peps.python.org/pep-3107/) with the following principles:
    + Annotations are completely optional
    + Python does not attach any particular meaning to annotations: just compile time associations of Python expressions to source code elements
- Later in [PEP 484](https://peps.python.org/pep-0484/) a syntax to use annotations for type checkers was standardized
    + A `typing` module was added to the standard library with a base set of specifications
    + Run-time behavior stays as before
    + It does NOT prevent other uses of annotations
    


## Python Annotations at Run-Time

- Run-time inspection of annotations ([Annotations Best Practices](https://docs.python.org/3/howto/annotations.html)):
    + Local variable annotations are not accessible at run-time
    ```python
    def f(a: float) -> int:
        result: int = int(a)  # this annotation is completely gone at run-time
        return result
    ```
    + Function, class and module annotations are available as `__annotations__`
        + Recommended way since Python 3.10 is to use `inspect.get_annotations()`
            + Unlike `typing.get_type_hints()`, it returns bare annotations
        + When `from __future__ import annotations` is used, annotations are stringized automatically by Python
       


In [1]:
import inspect

def f(a: float) -> int:
    result: int = int(a)  # this annotation is completely gone at run-time
    return result

class A:
    foo: int
        
    def method(a: int) -> str:
        return str(a)
    
print(f"{f.__annotations__ = }")
print(f"{inspect.get_annotations(A) = }")
print(f"{A.method.__annotations__ = }")


f.__annotations__ = {'a': <class 'float'>, 'return': <class 'int'>}
inspect.get_annotations(A) = {'foo': <class 'int'>}
A.method.__annotations__ = {'a': <class 'int'>, 'return': <class 'str'>}


In [2]:
from __future__ import annotations

import inspect

def f(a: float) -> int:
    result: int = int(a)  # this annotation is completely gone at run-time
    return result

class A:
    foo: int
        
    def method(a: int) -> str:
        return str(a)
    
print(f"{f.__annotations__ = }")
print(f"{inspect.get_annotations(A) = }")
print(f"{A.method.__annotations__ = }")


f.__annotations__ = {'a': 'float', 'return': 'int'}
inspect.get_annotations(A) = {'foo': 'int'}
A.method.__annotations__ = {'a': 'int', 'return': 'str'}


## Stringized Python Annotations at Run-Time

- Using `from __future__ import annotations` is preferred
    + It avoids automatic run-time evaluation of the expressions when loading the module
    + It avoids problem with symbols defined later
    ```python
    from __future__ import annotations  # This is required

    class A:
        parent: B  # without future import, this should be 'B'
        children: list[A] # without future import, this should be list['A']

    class B:
        pass
    ```
    

- but it has some issues...
    - Strings need to be evaluated to get its actual value
    ```python
    eval(f.__annotations__['a'])
    ```
    - When using annotations as typing hints, `typing.get_type_hints()` should be used:
    ```python
    typing.get_type_hints(f)['a']
    ```

In [3]:
from __future__ import annotations

import inspect
import typing

def f(a: float) -> int:
    result: int = int(a)
    return result

print(f"{inspect.get_annotations(f) = }")
print(f"{typing.get_type_hints(f) = }")

inspect.get_annotations(f) = {'a': 'float', 'return': 'int'}
typing.get_type_hints(f) = {'a': <class 'float'>, 'return': <class 'int'>}


## get_type_hints(obj, globalns=None, localns=None, include_extras=False)

- Basically, `get_type_hints()` evaluates the stringized annotations passing the expected _global_ and _local_ namespaces to `eval()`:
    + If `o` is a module, it uses `o.__dict__` as the globals when calling `eval()`
    + If `o` is a class, it uses `sys.modules[o.__module__].__dict__` as the globals, and `dict(vars(o))` as the locals, when calling `eval()`
    + If `o` is a wrapped callable using `functools.update_wrapper()`, `functools.wraps()`, or `functools.partial()`, it iteratively unwraps it by accessing either `o.__wrapped__` or `o.func` as appropriate, until finding the root unwrapped function
    + If `o` is a callable (but not a class), it uses `o.__globals__` as the globals when calling `eval()`
- Note that _nonlocal_ variables will not work by default
    ```python
    def f(a: float) -> int:
        Foo = int
        def inner(inner_a: Foo) -> None:
            pass
    ```

In [5]:
from __future__ import annotations

import typing

class C:
    class InnerC:
        pass
    
    def inner(inner_a: InnerC) -> None:
        pass
    
#print(f"{typing.get_type_hints(C.inner) = }")
print(f"{typing.get_type_hints(C.inner, localns={'InnerC': float}) = }")


typing.get_type_hints(C.inner, localns={'InnerC': float}) = {'inner_a': <class 'float'>, 'return': <class 'NoneType'>}


## Generic Aliases

- `typing` defines some parametrized constructs to specify typings other than just types (`Optional`, `Union`, `Generic`, `TypeVar`, ...)
- At run-time they are just _dumb_ objects and their values should be considered private implementation details of the `typing` module
- Exception: _concrete_ instantiation of generic types return `types.GenericAlias` instances which can be used as
    + instance constructors (discouraged): `list[int]() == list()`
    + base classes:
    ```python
    class A(list[int]):
        pass
    ```   
    + In both cases, they are **completely equivalent** to use the generic type
    + Before standard types were made generic, aliases existed in `typing` were required (`typing.List[int]`), and those return a private class from `typing` instead of `types.GenericAlias`


In [5]:
import types
import typing

list[int]() == list()

class A(list[int]):
    pass

print(f"{A.__mro__ = }\n")

print(f"{type(list[int]) = }")
print(f"{isinstance(list[int], types.GenericAlias) = }")
print(f"{isinstance(list[int], type) = }\n")

print(f"{type(typing.List[int]) = }")
print(f"{isinstance(typing.List[int], types.GenericAlias) = }")
print(f"{isinstance(typing.List[int], type) = }")


A.__mro__ = (<class '__main__.A'>, <class 'list'>, <class 'object'>)

type(list[int]) = <class 'types.GenericAlias'>
isinstance(list[int], types.GenericAlias) = True
isinstance(list[int], type) = True

type(typing.List[int]) = <class 'typing._GenericAlias'>
isinstance(typing.List[int], types.GenericAlias) = False
isinstance(typing.List[int], type) = False


## Bonus: stringized Python Annotations future?

- Stringized Pythons were supposed to be standard behavior since 3.10
    - `from __future__ import annotations` was introduced as a backport to use them from previous versions
- Last minute before Python 3.10 was released, the decision was changed due to heavy complaints from creators of libraries heavily using annotations at run-time (e.g. `pydantic`)
- Another proposal was born in the meantime to try to address all issues and it has been finally approved for python 3.13:
    https://peps.python.org/pep-0649/
